**Task 07: Querying RDF(s)**

In [17]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Read the RDF file

In [18]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N50fac9d95bf14303b685fdef686f0aeb (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

If we just want the "direct" subclasses of "Person":

RDFLib

In [19]:
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)

http://somewhere#Researcher


RDFLib and SPARQL

In [20]:
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#Researcher'),)


If we want all the subclasses, including the subclasses of the subclasses:

In [21]:
q2 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s rdfs:subClassOf/rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q2):
  print(r)

(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


RDFLib

In [22]:
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
subclass = g.value(subject=None, predicate = RDFS.subClassOf, object=ns.Person) #Person has just one subclass
print(g.value(subject=None, predicate=RDF.type, object=subclass)) #The subclass of Person has just one instance

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


RDFLib and SPARQL

In [23]:
q3 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q3):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


RDFLib

In [24]:
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for x,y,z in g.triples((s, None, None)):
    print(s, y)
subclass = g.value(subject=None, predicate = RDFS.subClassOf, object=ns.Person) #Person has just one subclass
subperson = g.value(subject=None, predicate=RDF.type, object=subclass) #The subclass of Person has just one instance
for x,y,z in g.triples((subperson, None, None)):
  print(subperson, y)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#typ

RDFLib and SPARQL

In [25]:
q4 = prepareQuery('''
  SELECT 
    ?s ?p
  WHERE { 
    ?s a/rdfs:subClassOf* ns:Person. 
    ?s ?p ?x
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q4):
  print(r.s, r.p)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#typ